In [22]:
# SOURCE: https://github.com/M-Taghizadeh/flan-t5-base-imdb-text-classification

In [ ]:
! pip install transformers pytorch_lightning sentencepiece datasets

In [ ]:
! pip install -q tqdm pandas scikit-learn evaluate nltk

In [ ]:
! pip install huggingface_hub accelerate

In [186]:
from huggingface_hub import login
login(token="hf_blabla")

In [3]:
import os
import shutil
import json
import time
import re
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [4]:
import datasets
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

UKRAINIAN_LETTERS = 'абвгґдеєжзиіїйклмнопрстуфхцчшщьюя'
UKRAINAIN_VOWELS = 'аеєиіїоуюя'
ENGLISH_LETTERS = 'abcdefghijklmnopqrstuvwxyz'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Preprocess Dataset

In [16]:
allowed_punctuation = """ .,!?;:'"«»()+-—–"""
other_punctuation =  """ $%&<>{}[]*"""

voa_df = pd.read_csv('./voa_stressed_cleaned_data.csv')
unique_letters = set(''.join(voa_df['text'].to_list()))

unique_letters = unique_letters - set(UKRAINIAN_LETTERS) \
                                - set(UKRAINIAN_LETTERS.upper()) \
                                - set(allowed_punctuation) \
                                - set(other_punctuation)

df = voa_df[~voa_df['text'].apply(lambda x: any(c in unique_letters for c in x))]
df = df[['text', 'labels']]
df = df.rename(columns={
    'labels': 'label'
})
df.shape

(137078, 2)

In [17]:
def get_data(df):
    train_df, eval_df = train_test_split(df, test_size=0.01, random_state=42)

    train_dataset = Dataset.from_pandas(train_df).remove_columns(['__index_level_0__'])
    eval_dataset = Dataset.from_pandas(eval_df).remove_columns(['__index_level_0__'])
    dataset = DatasetDict({
        "train": train_dataset,
        "eval": eval_dataset
    })
    return dataset


model_id = 'google/byt5-small'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
dataset = get_data(df)

In [18]:
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["eval"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])

tokenized_targets = concatenate_datasets([dataset["train"], dataset["eval"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])

Map:   0%|          | 0/137078 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/137078 [00:00<?, ? examples/s]

In [108]:
max_source_length, max_target_length

(1364, 1467)

In [19]:
def preprocess_function(sample, padding="max_length"):
    # print(sample)
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["label"] = labels["input_ids"]
    return model_inputs

In [20]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])

Map:   0%|          | 0/135707 [00:00<?, ? examples/s]

Map:   0%|          | 0/1371 [00:00<?, ? examples/s]

In [55]:
tokenized_dataset.save_to_disk("tokenized_voa_dataset")

Saving the dataset (0/6 shards):   0%|          | 0/135707 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1371 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = load_from_disk("tokenized_voa_dataset")

# Setup metrics

In [205]:
import re
import numpy as np

UKRAINAIN_VOWELS = 'аеєиіїоуюя'

def compute_stress_metrics(eval_preds):
    global UKRAINAIN_VOWELS, tokenizer
    
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    def split_with_stress(sentence):
        return re.split(r'(\s+|-)', sentence)

    match_results = []
    for pred, label in zip(decoded_preds, decoded_labels):
        pred_split = split_with_stress(pred)
        label_split = split_with_stress(label)

        if len(pred_split) == len(label_split):
            matches = []
            for p, l in zip(pred_split, label_split):
                if not any(char in UKRAINAIN_VOWELS for char in l):
                    continue
                if p==l:
                    matches.append(1)
                elif p.replace('+', '') != l.replace('+', ''):
                    matches = [0]
                    break
                else:
                    matches.append(0)
            if not matches: matches=[0]
            match_results.append(np.mean(matches))
        else:
            match_results.append(0) 

    # average match score across
    result = {"average_match": np.mean(match_results) * 100}

    return result

In [206]:
eval_preds = (
    [
        "Тр+идцять р+оків т+ому.",
        "Тр+идцять р+оків том+у.",
        "Тридцять р+оків том+у.",
        "Тридц+ять рок+ів том+у.",  # 0, cause all wrong stresses
        "Тр+идцять р+оків !", # 0, cause wrong words
        "Тр+идцять р+оків", # 0, cause wrong words
    ],
    [
        "Тр+идцять р+оків т+ому.",
        "Тр+идцять р+оків т+ому.",
        "Тр+идцять р+оків т+ому.",
        "Тр+идцять р+оків т+ому.",
        "Тр+идцять р+оків т+ому.",
        "Тр+идцять р+оків т+ому.",
    ]
)
for pred, label in zip(eval_preds[0], eval_preds[1]):
    encoded_pred = tokenizer(pred, padding=True, truncation=True, return_tensors="pt")["input_ids"]
    encoded_label = tokenizer(label, padding=True, truncation=True, return_tensors="pt")["input_ids"]
    result = compute_stress_metrics((encoded_pred, encoded_label))
    print(result)
    

print("Average: ")
encoded_preds = tokenizer(eval_preds[0], padding=True, truncation=True, return_tensors="pt")["input_ids"]
encoded_labels = tokenizer(eval_preds[1], padding=True, truncation=True, return_tensors="pt")["input_ids"]
result = compute_stress_metrics((encoded_preds, encoded_labels))
result

{'average_match': 100.0}
{'average_match': 66.66666666666666}
{'average_match': 33.33333333333333}
{'average_match': 0.0}
{'average_match': 0.0}
{'average_match': 0.0}
Average: 


{'average_match': 33.33333333333333}

# Setup training

In [213]:
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-accentor-model"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=False,
    
    learning_rate=3e-4,
    num_train_epochs=2,
    # max_steps=5000,
    
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    eval_strategy="steps",
    eval_steps=2000,

    save_total_limit=2,
    save_strategy="epoch",
    save_steps=500,

    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    # report_to="tensorboard",

    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"].select(range(500)),
    compute_metrics=compute_stress_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [179]:
import torch
torch.cuda.empty_cache()

# Evaluate

## Evaluate custom text

In [111]:
def generate_prediction(input_text, model, tokenizer, device='cuda'):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1500)

    decoded_preds = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded_preds

In [113]:
input_text = "Привіт, як справи?"

decoded_prediction = generate_prediction(input_text, model, tokenizer)
decoded_prediction

' Прив+іт, +як спр+ави?'

In [118]:
input_text = "відкрити ключем замок"

decoded_prediction = generate_prediction(input_text, model, tokenizer)
decoded_prediction

'відкр+ити кл+ючем з+амок'

In [114]:
input_text = "Сидить бобер на березі"

decoded_prediction = generate_prediction(input_text, model, tokenizer)
decoded_prediction

' С+идить б+обер н+а б+ерезі'

In [116]:
input_text = "Листя на березі"

decoded_prediction = generate_prediction(input_text, model, tokenizer)
decoded_prediction

' Лист+я н+а б+ерезі'

# Get metrics

In [177]:
from IPython.display import display, HTML
model.eval()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

all_predictions = []
i = 0

with torch.no_grad():
    for batch in tqdm(tokenized_dataset["eval"]):
        i += 1
        if i==100: break

        input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).unsqueeze(0).to(device)
        
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1500)
        decoded_preds = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        labels = batch["label"]
        labels = [l for l in labels if l != -100]
        decoded_labels = tokenizer.decode(labels, skip_special_tokens=True)
        decoded_inputs = tokenizer.decode(batch["input_ids"], skip_special_tokens=True)
        
        all_predictions.append((decoded_inputs, decoded_preds, decoded_labels))

df_predictions = pd.DataFrame(all_predictions, columns=["Input Text", "Predictions", "True Labels"])    

  7%|▋         | 99/1371 [00:42<09:02,  2.34it/s]


In [178]:
def highlight_differences(prediction, label):
    pred_words = re.split(r'(\s+|-)', prediction)
    label_words = re.split(r'(\s+|-)', label)

    highlighted_pred = []
    highlighted_label = []

    for pred_word, label_word in zip(pred_words, label_words):
        if pred_word != label_word:
            highlighted_label.append(f'<span style="color: red; font-weight: bold;">{label_word}</span>')
            highlighted_pred.append(f'<span style="color: red; font-weight: bold;">{pred_word}</span>')
        else:
            highlighted_label.append(label_word)
            highlighted_pred.append(pred_word)

    # Join the lists back into strings
    highlighted_pred = ''.join(highlighted_pred)
    highlighted_label = ''.join(highlighted_label)
    return highlighted_pred, highlighted_label

highlighted_preds = []
highlighted_labels = []
df_pred_highlighted = df_predictions.copy()

for pred, label in zip(df_pred_highlighted["Predictions"], df_pred_highlighted["True Labels"]):
    highlighted_pred, highlighted_label = highlight_differences(pred, label)
    highlighted_preds.append(highlighted_pred)
    highlighted_labels.append(highlighted_label)

df_pred_highlighted["Predictions"] = highlighted_preds
df_pred_highlighted["True Labels"] = highlighted_labels

display(HTML(df_pred_highlighted.head(10).to_html(escape=False)))

,Input Text,Predictions,True Labels
0,Тридцять років тому.,Тр+идцять р+оків том+у.,Тр+идцять р+оків т+ому.
1,Потім було музичне училище по класу ударних інструментів.,П+отім бул+о муз+ичне уч+илище п+о кл+асу уд+арних інструм+ентів.,П+отім бул+о муз+ичне уч+илище п+о кл+асу уд+арних інструм+ентів.
2,але вона може займатися.,ал+е вон+а м+оже займ+атися.,ал+е вон+а м+оже займ+атися.
3,"Цю операцію потрібно провести якомога швидше. На цьому наголошують як українська влада, так і місцеві мешканці, яким вдалося виїхати з Маріуполя.","Ц+ю опер+ацію потр+ібно провест+и яком+ога шв+идше. Н+а цьом+у нагол+ошують +як укра+їнська вл+ада, т+ак +і місц+еві м+ешканці, як+им вдал+ося в+иїхати з Марі+уполя.","Ц+ю опер+ацію потр+ібно провест+и яком+ога шв+идше. Н+а цьом+у нагол+ошують +як укра+їнська вл+ада, т+ак +і місц+еві м+ешканці, як+им вдал+ося в+иїхати з Марі+уполя."
4,Тим часом Литва наказала своїм урядовцям позбутися телефонів к Сайомі та Хуавей.,Т+им ч+асом Литв+а наказ+ала сво+їм уряд+овцям позб+утися телеф+онів к Сай+омі т+а Хуав+ей.,Т+им ч+асом Литв+а наказ+ала св+оїм уряд+овцям позб+утися телеф+онів к Сай+омі т+а Хуав+ей.
5,пів години обговорював перспективи вступу україни в нато з президентом сша джо Байденом,п+ів год+ини обгов+орював перспект+иви вст+упу укра+їни в н+ато з презид+ентом сш+а дж+о Байд+еном,п+ів год+ини обгов+орював перспект+иви вст+упу укра+їни в н+ато з презид+ентом сш+а дж+о Б+айденом
6,"І також традиційно виголосив промову, звернувся.","+І т+акож традиц+ійно виголос+ив пром+ову, зверн+увся.","+І т+акож традиц+ійно в+иголосив пром+ову, зверн+увся."
7,"пояснює, що це все однозначно спричинене людською діяльністю.","по+яснює, щ+о ц+е вс+е однозн+ачно сприч+инене л+юдською д+іяльністю.","по+яснює, щ+о ц+е вс+е однозн+ачно сприч+инене л+юдською ді+яльністю."
8,"І Джо Байден пообіцяв, що всі, хто хоче виїхати, виїдуть.","+І Дж+о Байд+ен пообіц+яв, щ+о вс+і, хт+о х+оче в+иїхати, в+иїдуть.","+І Дж+о Б+айден пообіц+яв, щ+о вс+і, хт+о х+оче в+иїхати, в+иїдуть."
9,"Юрій Мамон, Євгенія Дюло, волос Америки, Каліфорнія.","+Юрій Мам+он, Євг+енія Д+юло, в+олос Ам+ерики, Каліф+орнія.","+Юрій Мам+он, Євг+енія Дюл+о, в+олос Ам+ерики, Каліф+орнія."
